# Can we see pulsations with this data?
Add phase column

In [11]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import numpy as np
from pathlib import Path
from gammapy.data import DataStore, EventList, Observation

In [12]:
import pint
pint.__version__

'0.9.8'

In [13]:
import pint.models as pmodels
from pint import toa

In [14]:
datastore = DataStore.from_dir("DL3_lstchain_v0.10.9/")
observations = datastore.get_observations(required_irf="point-like")

In [15]:
target_pos = SkyCoord(
    ra=083.63311, dec=+22.014487, unit="deg", frame="icrs"
)

## Check the emphemeris 

Download from https://www.slac.stanford.edu/~kerrm/fermi_pulsar_timing/

In [38]:
ephemeris_file = "J0534+2200_54683_56587_chol.par.txt"

In [39]:
model = pmodels.get_model(ephemeris_file)


2024-05-13 15:26:40.351 | INFO     | pint.models.absolute_phase:validate:74 - TZRFRQ was 0.0 or None. Setting to infinite frequency.
2024-05-13 15:26:40.352 | WARNING  | pint.models.model_builder:__call__:202 - UNITS is not specified. Assuming TDB...


In [40]:
print(model.components["AstrometryEquatorial"])
print(model.components["SolarSystemShapiro"])
print(model.components["DispersionDM"])
print(model.components["AbsPhase"])
print(model.components["Spindown"])

AstrometryEquatorial(
    MJDParameter(   POSEPOCH            50739.0000000000000000 (d) frozen=True),
    floatParameter( PX                  0.0               (mas) frozen=True),
    AngleParameter( RAJ                 5:34:31.94000000  (hourangle) frozen=True),
    AngleParameter( DECJ                22:00:52.10000000 (deg) frozen=True),
    floatParameter( PMRA                -11.8             (mas / yr) +/- 1.0 mas / yr frozen=True),
    floatParameter( PMDEC               4.4               (mas / yr) +/- 1.0 mas / yr frozen=True))
SolarSystemShapiro(
    boolParameter(  PLANET_SHAPIRO      N                 frozen=True))
DispersionDM(
    floatParameter( DM                  56.78557939758982 (pc / cm3) frozen=True),
    floatParameter( DM1                 0.03127916834977064 (pc / (yr cm3)) frozen=True),
    MJDParameter(   DMEPOCH             55107.8071585532816240 (d) frozen=True))
AbsPhase(
    MJDParameter(   TZRMJD              55638.1552775999516551 (d) frozen=True),
    st

In [102]:
for obs in observations:
    print("Extracting phases for obs id: ", obs.obs_id)
    times = obs.events.time
    include_bipm = False
    include_gps = False

    # Set this to True or False depending on your ephemeris file.
    # Here we can see that the 'PLANET_SHAPIRO' entry is 'N' so we set it to True.
    planets = False
    
    # Create a TOA object for each time
    toas = toa.get_TOAs_array(
        times=times,
        obs="LST",
        errors=1 * u.microsecond,
        ephem="DE421",
        include_gps=include_gps,
        include_bipm=include_bipm,
        planets=planets,
    )
    phases = model.phase(toas, abs_phase=True)[1]

    # Shift phases to the interval (0,1]
    phases = np.where(phases < 0.0, phases + 1.0, phases)

    # Add phase column to table

    table = obs.events.table
    table["PHASE"] = phases.astype("float64") 

    new_event_list = EventList(table)
    new_event_list.write(filename="DL3_lstchain_v0.10.9/" + f"dl3_pulsar_{obs.obs_id:04d}.fits.gz", overwrite=True)
    

Extracting phases for obs id:  7253


2024-05-14 17:58:59.179 | DEBUG    | pint.toa:__init__:1358 - No pulse number flags found in the TOAs
2024-05-14 17:59:00.428 | DEBUG    | pint.toa:apply_clock_corrections:2200 - Applying clock corrections (include_gps = False, include_bipm = False)
2024-05-14 18:00:51.381 | INFO     | pint.observatory.topo_obs:clock_corrections:365 - Observatory lst requires no clock corrections.
2024-05-14 18:05:13.501 | DEBUG    | pint.toa:compute_TDBs:2251 - Computing TDB columns.
2024-05-14 18:05:13.502 | DEBUG    | pint.toa:compute_TDBs:2272 - Using EPHEM = DE421 for TDB calculation.
2024-05-14 18:07:16.923 | DEBUG    | pint.toa:compute_posvels:2350 - Computing PosVels of observatories and Earth, using DE421
2024-05-14 18:07:38.711 | INFO     | pint.solar_system_ephemerides:_load_kernel_link:55 - Set solar system ephemeris to de421 from download
2024-05-14 18:08:32.836 | DEBUG    | pint.toa:compute_posvels:2403 - SSB obs pos [-1.43878187e+11  3.80219368e+10  1.65137374e+10] m
2024-05-14 18:08:33.

Extracting phases for obs id:  7254


2024-05-14 18:09:54.689 | DEBUG    | pint.toa:__init__:1358 - No pulse number flags found in the TOAs
2024-05-14 18:09:55.849 | DEBUG    | pint.toa:apply_clock_corrections:2200 - Applying clock corrections (include_gps = False, include_bipm = False)
2024-05-14 18:11:08.734 | INFO     | pint.observatory.topo_obs:clock_corrections:365 - Observatory lst requires no clock corrections.
2024-05-14 18:15:06.756 | DEBUG    | pint.toa:compute_TDBs:2251 - Computing TDB columns.
2024-05-14 18:15:06.756 | DEBUG    | pint.toa:compute_TDBs:2272 - Using EPHEM = DE421 for TDB calculation.
2024-05-14 18:17:09.235 | DEBUG    | pint.toa:compute_posvels:2350 - Computing PosVels of observatories and Earth, using DE421
2024-05-14 18:17:29.794 | INFO     | pint.solar_system_ephemerides:_load_kernel_link:55 - Set solar system ephemeris to de421 from download
2024-05-14 18:18:19.249 | DEBUG    | pint.toa:compute_posvels:2403 - SSB obs pos [-1.43891858e+11  3.79823148e+10  1.64965974e+10] m
2024-05-14 18:18:20.

Extracting phases for obs id:  7255


...

2024-05-14 18:34:48.934 | DEBUG    | pint.toa:__init__:1358 - No pulse number flags found in the TOAs
2024-05-14 18:34:49.909 | DEBUG    | pint.toa:apply_clock_corrections:2200 - Applying clock corrections (include_gps = False, include_bipm = False)
2024-05-14 18:35:35.016 | INFO     | pint.observatory.topo_obs:clock_corrections:365 - Observatory lst requires no clock corrections.
2024-05-14 18:38:55.472 | DEBUG    | pint.toa:compute_TDBs:2251 - Computing TDB columns.
2024-05-14 18:38:55.473 | DEBUG    | pint.toa:compute_TDBs:2272 - Using EPHEM = DE421 for TDB calculation.
2024-05-14 18:40:22.226 | DEBUG    | pint.toa:compute_posvels:2350 - Computing PosVels of observatories and Earth, using DE421
2024-05-14 18:40:39.009 | INFO     | pint.solar_system_ephemerides:_load_kernel_link:55 - Set solar system ephemeris to de421 from download
2024-05-14 18:41:20.887 | DEBUG    | pint.toa:compute_posvels:2403 - SSB obs pos [-1.44640257e+11  3.56436381e+10  1.54828477e+10] m
2024-05-14 18:41:21.

Extracting phases for obs id:  7276


2024-05-14 18:41:46.764 | DEBUG    | pint.toa:__init__:1358 - No pulse number flags found in the TOAs
2024-05-14 18:41:47.066 | DEBUG    | pint.toa:apply_clock_corrections:2200 - Applying clock corrections (include_gps = False, include_bipm = False)
2024-05-14 18:42:16.461 | INFO     | pint.observatory.topo_obs:clock_corrections:365 - Observatory lst requires no clock corrections.
2024-05-14 18:43:18.821 | DEBUG    | pint.toa:compute_TDBs:2251 - Computing TDB columns.
2024-05-14 18:43:18.822 | DEBUG    | pint.toa:compute_TDBs:2272 - Using EPHEM = DE421 for TDB calculation.
2024-05-14 18:43:46.276 | DEBUG    | pint.toa:compute_posvels:2350 - Computing PosVels of observatories and Earth, using DE421
2024-05-14 18:43:51.682 | INFO     | pint.solar_system_ephemerides:_load_kernel_link:55 - Set solar system ephemeris to de421 from download
2024-05-14 18:44:04.728 | DEBUG    | pint.toa:compute_posvels:2403 - SSB obs pos [-1.44651225e+11  3.56095041e+10  1.54681383e+10] m
2024-05-14 18:44:05.

Extracting phases for obs id:  7277


2024-05-14 18:44:35.075 | DEBUG    | pint.toa:__init__:1358 - No pulse number flags found in the TOAs
2024-05-14 18:44:35.809 | DEBUG    | pint.toa:apply_clock_corrections:2200 - Applying clock corrections (include_gps = False, include_bipm = False)
2024-05-14 18:45:07.389 | INFO     | pint.observatory.topo_obs:clock_corrections:365 - Observatory lst requires no clock corrections.
2024-05-14 18:47:41.513 | DEBUG    | pint.toa:compute_TDBs:2251 - Computing TDB columns.
2024-05-14 18:47:41.514 | DEBUG    | pint.toa:compute_TDBs:2272 - Using EPHEM = DE421 for TDB calculation.
2024-05-14 18:48:37.441 | DEBUG    | pint.toa:compute_posvels:2350 - Computing PosVels of observatories and Earth, using DE421
2024-05-14 18:48:49.998 | INFO     | pint.solar_system_ephemerides:_load_kernel_link:55 - Set solar system ephemeris to de421 from download
2024-05-14 18:49:21.683 | DEBUG    | pint.toa:compute_posvels:2403 - SSB obs pos [-1.44656305e+11  3.55936456e+10  1.54613108e+10] m
2024-05-14 18:49:22.

In [93]:
new_hdu = datastore.hdu_table.copy()
output_directory = "./pulsar_events_file/"
for entry in new_hdu:
    for obs in observations:
        if entry["HDU_TYPE"] == "events" and entry["OBS_ID"] == obs.obs_id:           
            entry["FILE_DIR"] = "./"
            entry["FILE_NAME"] =  f"dl3_pulsar_{obs.obs_id:04d}.fits.gz"
            print(entry["FILE_DIR"], entry["FILE_NAME"])

. dl3_pulsar_7253.fits.gz
. dl3_pulsar_7254.fits.gz
. dl3_pulsar_7255.fits.gz
. dl3_pulsar_7256.fits.gz
. dl3_pulsar_7274.fits.gz
. dl3_pulsar_7275.fits.gz
. dl3_pulsar_7276.fits.gz
. dl3_pulsar_7277.fits.gz


In [94]:
output_directory

'./pulsar_events_file/'

In [95]:
new_hdu

OBS_ID,HDU_TYPE,HDU_CLASS,FILE_DIR,FILE_NAME,HDU_NAME,SIZE
int64,bytes8,bytes10,bytes1,bytes23,bytes17,int64
7253,events,events,.,dl3_pulsar_7253.fits.gz,EVENTS,148884480
7253,gti,gti,.,dl3_LST-1.Run07253.fits,GTI,148884480
7253,pointing,pointing,.,dl3_LST-1.Run07253.fits,POINTING,148884480
7253,aeff,aeff_2d,.,dl3_LST-1.Run07253.fits,EFFECTIVE AREA,148884480
7253,edisp,edisp_2d,.,dl3_LST-1.Run07253.fits,ENERGY DISPERSION,148884480
7253,rad_max,rad_max_2d,.,dl3_LST-1.Run07253.fits,RAD_MAX,148884480
...,...,...,...,...,...,...
7276,rad_max,rad_max_2d,.,dl3_LST-1.Run07276.fits,RAD_MAX,35602560
7277,events,events,.,dl3_pulsar_7277.fits.gz,EVENTS,88323840


In [96]:
new_hdu.write(
    datastore_dir + "hdu-index-pulsar.fits.gz", format="fits", overwrite=True
)

In [98]:
new_obs.write?

Signature:
new_obs.write(
    path,
    overwrite=False,
    format='gadf',
    include_irfs=True,
    checksum=False,
)
Docstring:
Write this observation into `~pathlib.Path` using the specified format.

Parameters
----------
path : str or `~pathlib.Path`
    Path for the output file.
overwrite : bool, optional
    Overwrite existing file. Default is False.
format : {"gadf"}
    Output format, currently only "gadf" is supported. Default is "gadf".
include_irfs : bool, optional
    Whether to include irf components in the output file. Default is True.
checksum : bool, optional
    When True adds both DATASUM and CHECKSUM cards to the headers written to the file.
    Default is False.
File:      ~/anaconda3/envs/gammapy-1.2/lib/python3.10/site-packages/gammapy/data/observations.py
Type:      method